# 5.3.1 The Validation Set Approach

In [1]:
import pandas as pd
import numpy as np

In [2]:
Auto = pd.read_csv('Data/Auto.csv', na_values='?').dropna()
Auto

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name
0,18.0,8,307.0,130.0,3504,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150.0,3436,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150.0,3433,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140.0,3449,10.5,70,1,ford torino
...,...,...,...,...,...,...,...,...,...
392,27.0,4,140.0,86.0,2790,15.6,82,1,ford mustang gl
393,44.0,4,97.0,52.0,2130,24.6,82,2,vw pickup
394,32.0,4,135.0,84.0,2295,11.6,82,1,dodge rampage
395,28.0,4,120.0,79.0,2625,18.6,82,1,ford ranger


In [3]:
import statsmodels.api as sm
from statsmodels.formula.api import ols
import statsmodels.formula.api as smf

fit1 = smf.ols('mpg~horsepower', data=Auto).fit()
(fit1.summary())

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    mpg   R-squared:                       0.606
Model:                            OLS   Adj. R-squared:                  0.605
Method:                 Least Squares   F-statistic:                     599.7
Date:                Sat, 10 Apr 2021   Prob (F-statistic):           7.03e-81
Time:                        17:47:23   Log-Likelihood:                -1178.7
No. Observations:                 392   AIC:                             2361.
Df Residuals:                     390   BIC:                             2369.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     39.9359      0.717     55.660      0.000      38.525      41.347
horsepower    -0.1578      0.006    -24.489      0.000      -0.171      -0.145
==============================================================================
Omnibus:                       16.432   Durbin-Watson:                   0.920
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               17.305
Skew:                           0.492   Prob(JB):                     0.000175
Kurtosis:                       3.299   Cond. No.                         322.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [4]:
from sklearn.metrics import mean_squared_error
y_pred = fit1.predict(Auto)
mean_squared_error(Auto['mpg'], y_pred)

23.943662938603108

The estimated error is 23.94 for the Linear Regression fit. Now we calculate the MSE for the quadratic and cubic regressions. 

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(Auto, Auto['mpg'], test_size=0.5, random_state=2)

formula = 'mpg~horsepower+I(horsepower**2)'
fit2 = smf.ols(formula, data=X_train).fit()
y_pred2 = fit2.predict(X_test)
mean_squared_error(y_test, y_pred2)

18.55019880190998

In [6]:
formula = 'mpg~horsepower+I(horsepower**3)'
fit3 = smf.ols(formula, data=X_train).fit()
y_pred3 = fit3.predict(X_test)
mean_squared_error(y_test, y_pred3)

18.902615007987375

The error rates for quadratic and cubic regression are 19.82 and 19.79 respectively. With different random states, we get different results as shown below. 

In [7]:
X_train, X_test, y_train, y_test = train_test_split(Auto, Auto['mpg'], test_size=0.5, random_state=2)

fit1 = smf.ols('mpg~horsepower', data=X_train).fit()
y_pred = fit1.predict(X_test)
print('Error for linear fit: ', mean_squared_error(y_test, y_pred))

formula = 'mpg~horsepower+I(horsepower**2)'
fit2 = smf.ols(formula, data=X_train).fit()
y_pred2 = fit2.predict(X_test)
print('Error for quadratic fit: ', mean_squared_error(y_test, y_pred2))

formula = 'mpg~horsepower+I(horsepower**3)'
fit3 = smf.ols(formula, data=X_train).fit()
y_pred3 = fit3.predict(X_test)
print('Error for quadratic fit: ', mean_squared_error(y_test, y_pred3))

Error for linear fit:  23.442643969985742
Error for quadratic fit:  18.55019880190998
Error for quadratic fit:  18.902615007987375


The results show that a non-linear fit performs better than a linear one. However, it is difficult to distinguish if the quadratic or cubic fit is better as the errors are very close.

# 5.3.2 Leave-One-Out Cross-Validation

In Leave-One-Out Cross-Validation (LOOCV), we calculate the error when we leave out one observation at a time. 
The LOOCV estimate for the test Mean Squared Error (MSE) is the average of these $n$ test error estimates and is given by $$CV_{(n)} = \frac{1}{n} \sum \limits_{i=1}^{n} MSE_{i}$$

When we use the Generalized Linear Model (GLM) function without passing Binomial to the family argument, it perfomrs linear regression.  

In [8]:
formula = 'mpg~horsepower'
glm = smf.glm(formula, data=Auto)
result = glm.fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                    mpg   No. Observations:                  392
Model:                            GLM   Df Residuals:                      390
Model Family:                Gaussian   Df Model:                            1
Link Function:               identity   Scale:                          24.066
Method:                          IRLS   Log-Likelihood:                -1178.7
Date:                Sat, 10 Apr 2021   Deviance:                       9385.9
Time:                        17:47:29   Pearson chi2:                 9.39e+03
No. Iterations:                     3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     39.9359      0.717     55.660      0.000      38.530      41.342
horsepower    -0.1578      0.006    -24.489      0.000      -0.170      -0.145
==============================================================================
"""

In [9]:
from sklearn import linear_model
from sklearn.model_selection import cross_val_score
lm = linear_model.LinearRegression(fit_intercept=True)
result = lm.fit(Auto['horsepower'].values.reshape(-1,1), Auto['mpg'].values.reshape(-1,1))
print('Coefficients: Intercept: ',result.intercept_)
print('Coefficients: horsepower: ',result.coef_)

Coefficients: Intercept:  [39.93586102]
Coefficients: horsepower:  [[-0.15784473]]


From the above results, it is evident that the glm model and the lm(linear regression) model have similar results. 
Performing cross validation using LOOCV on the linear regression model (Cross validation using the statsmodel api would require a [wrapper](https://stackoverflow.com/questions/41045752/using-statsmodel-estimations-with-scikit-learn-cross-validation-is-it-possible) class):

In [10]:

from sklearn.model_selection import LeaveOneOut
loo = LeaveOneOut()

X = Auto['horsepower'].values.reshape(-1,1)
y = Auto['mpg'].values.reshape(-1,1)
loo.get_n_splits(X)

392

As we can see there are 392 splits, i.e $n$ splits on thet data when we use LOOCV. We use the `cross_val_score` function to evaluate the results.

In [11]:
from sklearn.model_selection import KFold

crossvalidation = KFold(n_splits=loo.get_n_splits(X), random_state=None, shuffle=False)
scores = cross_val_score(result, X, y, scoring="neg_mean_squared_error", cv=crossvalidation,
 n_jobs=1)
print("Folds: " + str(len(scores)) + ", MSE: " + str(np.mean(np.abs(scores))) + ", STD: " + str(np.std(scores)))

Folds: 392, MSE: 24.231513517929226, STD: 36.79731503640535


The cross validation estimate for the error of the linear model is 24.23.
We can also estimate the error for increasingly complex polynomial fits. 

Since `cross_val_score` is supported only by `sklearn` and not `statsmodels`, I transform the data into non-linear values using the `PolynomialFeatures` function from `sklearn`. Note that when I used non-linear transformations of th data using the `statsmodel` library, there was no need for an additional function, I only needed to define my polynomial term inside `I()` with the corresponding degree of polynomial.

In [12]:
from sklearn.preprocessing import PolynomialFeatures
for i in range(1,6):
    poly = PolynomialFeatures(degree=i)
    X_current = poly.fit_transform(X)
    result = lm.fit(X_current, y)
    scores = cross_val_score(result, X_current, y, scoring="neg_mean_squared_error", cv=crossvalidation,
 n_jobs=1)
    
    print("Degree-"+str(i)+" polynomial MSE: " + str(np.mean(np.abs(scores))) + ", STD: " + str(np.std(scores)))

Degree-1 polynomial MSE: 24.231513517929226, STD: 36.79731503640535
Degree-2 polynomial MSE: 19.24821312448939, STD: 34.998446151782474
Degree-3 polynomial MSE: 19.334984064114092, STD: 35.765135678007795
Degree-4 polynomial MSE: 19.424430309411886, STD: 35.68335275769751
Degree-5 polynomial MSE: 19.033211842978396, STD: 35.31729288251292


The difference between linear and polynomial fits are clearly see in the MSE.

# 5.3.3 k-Fold Cross-Validation

The k-fold cross validation is similar to the Leave-One-Out-Cross-Validation, except that instead of leaving out one observation at a time, we leave out k observations at a time which serve as the test set. 

There exists a bias-variance tradeoff with the choice of k in kCV. The LOOCV method, where `k=1` has low bias but higher variance compared to the kCV method. 
`k=5` or `k=10` are the commonly used values as it has been proven empirically to yield test error estimates that do not have excessively high bias or variance.  

In [13]:
crossvalidation = KFold(n_splits=10,shuffle=False)

for i in range(1,11):
    poly = PolynomialFeatures(degree=i)
    X_current = poly.fit_transform(X)
    model = lm.fit(X_current, y)
    scores = cross_val_score(model, X_current, y, scoring="neg_mean_squared_error", cv=crossvalidation, n_jobs=1)
    print("Degree-"+str(i)+" polynomial MSE: " + str(np.mean(np.abs(scores))) + ", STD: " + str(np.std(scores)))

Degree-1 polynomial MSE: 27.439933652339857, STD: 14.510250711281133
Degree-2 polynomial MSE: 21.235840055802118, STD: 11.797327528898292
Degree-3 polynomial MSE: 21.336606183328417, STD: 11.8443397146369
Degree-4 polynomial MSE: 21.353886994209773, STD: 11.986332342224673
Degree-5 polynomial MSE: 20.905646119059934, STD: 12.18560440073758
Degree-6 polynomial MSE: 20.82189095906726, STD: 12.126258882595026
Degree-7 polynomial MSE: 20.953103378424785, STD: 12.059908521569556
Degree-8 polynomial MSE: 21.077131510426256, STD: 12.04447106023584
Degree-9 polynomial MSE: 21.03675183384266, STD: 11.948760351967676
Degree-10 polynomial MSE: 20.981013741561554, STD: 11.797365253121383


The computation time is shorter for kCV as compared to LOOCV. 

# 5.3.4 The Bootstrap

In this method we repeatedly sample observations from the original dataset to obtain distinct datasets. The sampling is performed with replacement - the same observation can occur more than once in the bootstrap dataset.

Bootstrapping does not assume the distribution of the data, it simply simulates multiple samples of the data with replacement. The required parameters, such as mean, variance are calcluated from the histogram of these values of each sample subset.

Suppose we have one predictor variable and two response variables $X$ and $Y$, and we choose a fraction $\alpha$ of the predictor variable for $X$ and $1-\alpha$ for $Y$. We want to minimize the total variance of $\alpha$, i.e $Var(\alpha*X + (1-\alpha)*Y)$. This value of alpha is given by:
 $$\alpha = \frac{\sigma_Y^2 - \sigma_{XY}}{\sigma_X^2 + \sigma_Y^2 - 2\sigma_{XY}}$$

where, $\sigma_X^2 = Var(X), \sigma_Y^2 = Var(Y), \sigma_XY = Cov(X,Y)$

We calculate estimates for each of the datasets and compute the standard error of all the estimates.

In [14]:
Portfolio = pd.read_csv('Data/Portfolio.csv')
Portfolio

,X,Y
0,-0.895251,-0.234924
1,-1.562454,-0.885176
2,-0.417090,0.271888
3,1.044356,-0.734198
4,-0.315568,0.841983
...,...,...
95,0.479091,1.454774
96,-0.535020,-0.399175
97,-0.773129,-0.957175
98,0.403634,1.396038


We define a function alpha that will calculate the alpha as in the equation shown above. 

In [15]:
# we use np.cov(X,Y)[0,1] not np.cov as the latter returns a matrix with the variance and covariance.
def alpha(X,Y):
    return (np.var(Y)-np.cov(X,Y)[0,1])/(np.var(X)+np.var(Y)-2*np.cov(X,Y)[0,1])

In [16]:
X = Portfolio.X[0:100]
Y = Portfolio.Y[0:100]
print(alpha(X,Y))

0.5766511516108046


Here we bootstrap the sample datasets used; we randomly select 100 observations with replacement from the dataset and calculate the $\alpha$ by recording all the estimates for it and calculating its mean. 


In [17]:
def bstrap(df):
    tresult = 0
    for i in range(0,100):
        dfsample = df.sample(frac=1, replace=True)
        X = dfsample.X[0:100]
        y = dfsample.Y[0:100]
        result = alpha(X,y)
        tresult += result
    fresult = tresult / 100
    print(fresult)
bstrap(Portfolio)

0.5716479267105111


#### Appendix

1. Bootstrapping explanation: https://statisticsbyjim.com/hypothesis-testing/bootstrapping/